This part of the pipeline processes the raw antiSMASH output and statistically compares the normalised BGC counts by rRNA cluster.

### Paths and parameters

#### Pipeline input folders

In [ ]:
metadata = "./genomes_metadata"

#### Pipeline output folders

In [ ]:
task_root = "./10-MGEs/BGCs"
output_folder = task_root+"/output"
results_folder = task_root+"/processed_output"

#### Tool pointers and parameters

#### Libraries and other setups

In [ ]:
from Bio import SeqIO
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools as it
import scipy.stats as sts
import numpy as np
from statannotations.Annotator import Annotator

In [ ]:
custom_palette = sns.husl_palette()
custom_palette = [custom_palette[0], custom_palette[2], custom_palette[4], custom_palette[5]]
custom_palette

In [ ]:
os.makedirs(results_folder, exist_ok=True)

## Reading input files

### Parsing the output

This parser will assign all hits to a certain BGC type so that we can differentiate these downstream, but this causes hybrid regions to be included multiple times, so we'll have to deduplicate these in the general count plots later on.

In [ ]:
os.makedirs(results_folder, exist_ok=True)
result_dirs = os.listdir(output_folder)
hits_hybrids = []
# Take a look into the result directory of each screened assembly
for dir in result_dirs:
    dir_conts = os.listdir(output_folder + '/' + dir)
    genbank_files = [f for f in dir_conts if '.region' in f]
    # Take a look into each GenBank region file
    for gbf in genbank_files:
        with open(output_folder + '/' + dir + '/' + gbf, "r") as handle:
            seq = list(SeqIO.parse(handle, 'genbank'))[0]
        # Get all product region tags in this file
        region_tags = [f for f in seq.features if f.type == "region"][0]
        types = region_tags.qualifiers['product']
        hybrid = len(types) != 1
        # Create a hit record for each BGC type in this region
        for type in types:
            length = int(region_tags.location.end)
            contig = seq.id
            region = int(region_tags.qualifiers['region_number'][0])
            record = {'assembly_ID': dir, 'contig_ID': contig, 'region': region, 'type': type, 'hybrid': hybrid, 'length': length}
            hits_hybrids.append(record)
hits_hybrids = pd.DataFrame(hits_hybrids)
hits_hybrids

In [ ]:
hits_hybrids.to_csv(results_folder + "/all_hits", sep = '\t', index = False)

### Loading the cluster annotations

In [ ]:
cluster_annotations_0 = pd.read_table(metadata, sep = '\t', usecols = [1,2])
cluster_annotations_0.columns = ['assemblyID', 'cluster']
cluster_annotations = cluster_annotations_0.to_dict(orient = 'list')
cluster_annotations = dict(zip(*cluster_annotations.values()))
cluster_annotations

### Reading the genome sizes

Necessary for normalising the general BGC counts

In [ ]:
genome_sizes_0 = pd.read_table(metadata, sep = '\t', usecols = [1,4])
genome_sizes_0.columns = ['assemblyID', 'size']

In [ ]:
genome_sizes = genome_sizes_0.to_dict(orient = 'list')
genome_sizes = dict(zip(*genome_sizes.values()))
genome_sizes

### Adding metadata

In [ ]:
hits_hybrids['cluster'] = hits_hybrids['assembly_ID'].apply(lambda x: cluster_annotations[x])
hits_hybrids['size'] = hits_hybrids['assembly_ID'].apply(lambda x: genome_sizes[x])
hits_hybrids

### General count plots

Let's now deduplicate the records of the hybrid regions after omitting the `hybrid` and `type` columns.

In [ ]:
hits = hits_hybrids.drop(columns = ['hybrid', 'type']).drop_duplicates()

Add metadata columns and normalise the general counts by genome size

In [ ]:
cluster_counts = pd.DataFrame(hits.groupby('assembly_ID')['contig_ID'].count()
                             ).reset_index().rename(columns = {'contig_ID': 'No. BGCs'})
cluster_counts['cluster'] = cluster_counts['assembly_ID'].apply(lambda x: cluster_annotations[x])
cluster_counts['size'] = cluster_counts['assembly_ID'].apply(lambda x: genome_sizes[x])
cluster_counts['Norm. no. BGCs'] = cluster_counts['No. BGCs']/cluster_counts['size']*1000000
cluster_counts

#### Barplot

In [ ]:
fig, ax = plt.subplots(figsize = (5,2))
ax = sns.barplot(ax = ax, data = cluster_counts, estimator = "mean", errorbar = "se",
                 x = "Norm. no. BGCs", y = "cluster", palette = custom_palette,
                 width = 0.9, orient = "h")
plt.xlabel('Avg. norm. no. BGCs')
plt.ylabel('rRNA cluster')
plt.title('BGCs')
plt.savefig(results_folder + "/" + "av_counts_BGCcluster_bar.svg")
plt.show()

#### Violinplot

In [ ]:
fig, ax = plt.subplots(figsize = (5,3))
ax = sns.violinplot(ax = ax, data = cluster_counts, x = 'Norm. no. BGCs', y = 'cluster',
                    palette = custom_palette, orient = 'h', cut = 0)
plt.xlabel('Norm. no. BGCs')
plt.ylabel('rRNA cluster')
plt.title('BGCs')

# Adding statistical significance marks
pairs = list(it.combinations(cluster_counts['cluster'].unique(), 2))
annotator = Annotator(ax = ax, pairs = pairs, data = cluster_counts, x = 'Norm. no. BGCs', y = 'cluster', orient = 'h', cut = 0)
annotator.configure(test = 'Mann-Whitney', text_format = 'star', loc = 'inside')
annotator.apply_and_annotate()

plt.savefig(results_folder + "/" + 'counts_BGCcluster_violin.svg')
plt.show()

#### Exact stats

Getting all counts grouped by rRNA cluster

In [ ]:
cluster_counts_stats = cluster_counts[['Norm. no. BGCs', 'cluster']].to_dict(orient = 'list')
cluster_counts_stats = list(zip(*cluster_counts_stats.values()))
counts_stats = {}
for record in cluster_counts_stats:
    try:
        counts_stats[record[1]].append(record[0])
    except KeyError:
        counts_stats[record[1]] = [record[0]]
counts_stats

In [ ]:
[(i, [np.mean(j), np.std(j)]) for i,j in counts_stats.items()]

In [ ]:
tests = it.combinations(counts_stats.keys(), 2)
for comb in tests:
    print(str(comb) + ': ' + 
          str(sts.mannwhitneyu(counts_stats[comb[0]], 
                           counts_stats[comb[1]])
              [1])
         )

### BGC types per cluster

Count by BGC type. Hybrids count for each region they're a hybrid of.

In [ ]:
type_counts = pd.DataFrame(hits_hybrids.groupby(['assembly_ID', 'type'])['contig_ID'].count()
                               ).reset_index().rename(columns = {'contig_ID': 'No. BGCs'})
type_counts

Readd rRNA cluster annotation and melt the dataframe by cluster annotation

In [ ]:
type_counts_pivot = type_counts.pivot(columns = "type", index = "assembly_ID", values = "No. BGCs").fillna(0).astype(int)
type_counts_pivot['cluster'] = type_counts_pivot.index.to_series().apply(lambda x: cluster_annotations[x])
type_counts_pivot = type_counts_pivot.melt(id_vars = 'cluster').rename(columns = {'value': 'No. BGCs'})
type_counts_pivot

In [ ]:
fig, ax = plt.subplots(figsize = (6,8))
ax = sns.barplot(ax = ax, data = type_counts_pivot, estimator = "mean", errorbar = "se",
                 x = "No. BGCs", y = "type", hue = "cluster", order = sorted(type_counts['type'].unique()),
                 palette = custom_palette, width = 0.9, orient = "h")
plt.xlabel('Avg. no. BGCs')
plt.ylabel('BGC class')
plt.savefig(results_folder + "/" + "av_counts_BGCtype.svg")
plt.show()